## Entraînement du modèle DELTA_DIST+H_05 (Réseau minimal + numéros atomiques)

L'entraînement des modèles précédents montre que la topologie des réseaux ne semble pas avoir une influence déterminante dans les performances. On a fait varier le nombre de couches cachées de 1 à 4, et la largueur des couches cachées de 1000 à 12000, et les performances ont toujours été sensiblement comparables. On a donc tendance à penser que l'on utilise des topologies trop complexes pour les tâches que nous tentons de réaliser.

Nous allons donc entraîner ici un réseau de taille plus raisonnable par rapport à la tâche que nous effectuons, dans l'idée que cela nous fera gagner beaucoup de temps de calcul et que nous pourrons donc effectuer des recherches par quadrillage des meilleurs hyper-paramètres beaucoup plus importantes.

Dans le même temps, nous avons dans l'idée qu'il manque peut-être aux réseaux l'information du numéro atomique des atomes pour effectuer des prédictions précises, le numéro atomique correspondant au nombre d'atomes d'hydrogène, qui ont une influence sur la géométrie de la molécule

Nous allons donc travailler sur le jeu de données réduit du notebook 8.1, contenant les entrées préparées (positions + masses + numéros atomiques) et les sorties (delta distances) pour toutes les molécules de tailles comprises entre 2 et 60.

Nous allons dans un premier temps entraîner un modèle avec des paramètres similaires à ceux utilisés dans les modèles précédents, puis analyser les résultats. 

Si les résultats sont identiques (une loss autour de 107), alors la réduction du jeu de données (notamment la suppression des données aberrantes), l'ajout des numéros atomiques et le travail sur un modèle contenant moins de connexions n'auront pas eu d'influence sur les résultats et nous allons pouvoir lancer des recherches par quadrillage beaucoup plus importantes des hyper-paramètres.

Si les résultats du modèle sont meilleurs, alors nous allons tenter d'ajuster encore mieux les hyper-paramètres et de rechercher quel facteur a causé l'amélioration des résultats.

Si les résultats du modèle sont pires, alors nous allons rechercher quel facteur a causé cette dégradation et l'ajuster.

### Résultats

Après avoir lancé l'entraînement du modèle sur 6e10 exemples (100 époques de 6 000 000 exemples), nous nous apercevons que les résultats sont très semblables aux modèles précédents, la losse stagne au bout de quelques batchs seulement autour de 106.8. Cela confirme que les modèles précédents étaient trop complexes. Nous avons donc l'avantage d'avoir un modèle beaucoup plus rapide à entraîner. Nous allons donc maintenant effectuer une recherche par quadrillage très importante des hyper-paramètres du modèle.


#### Chemin des fichiers

In [8]:
train_prepared_input_loc = "../data/train_set_riken_v2_prepared_input_bruit+_anums_reduced.h5"
train_labels_loc = "../data/train_set_riken_v2_labels_bruit+_anums_reduced.h5"

mini_prepared_input_loc = "../data/mini_set_prepared_input_bruit+_anums_reduced.h5"
mini_labels_loc = "../data/mini_set_labels_bruit+_anums_reduced.h5"

models_loc = "../models/DELTA_DIST+H_05/8.3/"
logs_loc = "../models/DELTA_DIST+H_05/8.3/"

## Création du RN

#### Fonction renvoyant le masque des atomes à prendre en compte pour les calculs

L'entrée et la sortie du RN étant définies par une méthode de padding, seul un certain nombre d'entrées et de sortie est utilisé pour chaque exemple d'entraînement en fonction du nombre d'atomes de la molécule. On définit ici une fonction qui renvoie le masque des différences de distances à prendre en compte sur les données en entrée et les étiquettes.

In [2]:
import tensorflow as tf

def calcul_masque_atomes_definis(targets):
    """ On calcule le masque booléen des atomes donnés en entrée du RN en fonction du vecteur targets"""
    
    # On cherche à obtenir un masque booléen des atomes définis en entrée. Pour cela, on prend en entrée
    # les étiquettes sous la forme d'une matrice (200, 4) dont chaque ligne i est la distance de l'atome i avec
    # les atomes fictifs du repère. L'atome est indéfini ssi. la somme de la ligne est nulle. En effet,
    # un atome défini ne peut pas avoir une distance nulle avec les quatre atomes fictifs, et on veille
    # à ce que le vecteurs targets ne contienne que des valeurs nulles pour les atomes non définis.
    # On obtient donc un masque booléen de tous les atomes définis en entrée
    
    ## On somme les distances de chaque atome ##
    targets_dists_sums = tf.reduce_sum(targets, 1)
    
    ## On créé le masque des sommes différentes de zéro ##
    
    # Création des matrice de True et de False de la dimension de la matrice des sommes (nécessaires
    # pour tf.where)
    zeros = tf.cast(tf.zeros_like(targets_dists_sums),dtype=tf.bool)
    ones = tf.cast(tf.ones_like(targets_dists_sums),dtype=tf.bool)
    
    return tf.where(targets_dists_sums>0, ones, zeros)


/home/etudiant/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/etudiant/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


#### Fonction de coût


In [10]:
def partial_rmse(predictions, targets):
    """ Calcule le RMSE partiel des prédictions par rapport aux valeurs attendues. Le RMSE est partiel car
    on ne le calcule que pour les sorties correspondant aux atomes donnés en entrée. En d'autres
    termes, on ne pousse pas le modèle à donner des distances nulles pour les atomes indéfinis
    en entrée"""
    
    with tf.name_scope("partial_rmse"):

        # On met les prédictions et les cibles sous la forme d'une matrice (200, 4)
        predictions = tf.reshape(predictions, [-1, 4])
        targets = tf.reshape(targets, [-1, 4])

        # On calcule le masque des atomes définis selon les cibles
        defined_atoms_mask = calcul_masque_atomes_definis(targets)
        
        # On masque les prédictions et les étiquettes selon le masque des atomes définis
        targets_masked = tf.boolean_mask(targets, defined_atoms_mask)
        predictions_masked = tf.boolean_mask(predictions, defined_atoms_mask)
        
        sess = tf.InteractiveSession()

        # Add print operation
        predictions_masked = tf.Print(predictions_masked, [predictions_masked], message="Predictions :")

        return tf.sqrt(tf.reduce_mean(tf.squared_difference(predictions_masked, targets_masked)), name="rmse")
    

#### Fonction d'évaluation des performances (score R2)

In [3]:
def partial_r2_score(predictions, targets, inputs):
    """ Renvoie le score R2 de la prédiction (le calcul est effectué uniquement sur les résultats
    des atomes donnés en entrée) """
    
    with tf.name_scope("partial_r2"):
    
        # On met les prédictions et les cibles sous la forme d'une matrice (200, 4)
        predictions = tf.reshape(predictions, [-1, 4])
        targets = tf.reshape(targets, [-1, 4])

        # On calcule le masque des atomes définis selon les cibles
        defined_atoms_mask = calcul_masque_atomes_definis(targets)

        # On masque les prédictions et les étiquettes selon le masque des atomes définis
        targets_masked = tf.boolean_mask(targets, defined_atoms_mask)
        predictions_masked = tf.boolean_mask(predictions, defined_atoms_mask)

        # Calcul de l'erreur totale
        total_error = tf.reduce_sum(tf.square(tf.subtract(targets, tf.reduce_mean(targets_masked))))

        # Calcul de l'erreur inexpliquée
        unexplained_error = tf.reduce_sum(tf.square(tf.subtract(targets_masked, predictions_masked)))

        r2 = tf.subtract(1.0, tf.divide(unexplained_error, total_error), "r2_score")
        return r2

#### Définition d'une fonction créant le RN

In [4]:
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.optimizers import Adam
from tflearn.data_preprocessing import DataPreprocessing
import math


def creer_360_x_240(epsilon=1e-8, learning_rate=0.001, dropout_val=0.99, stddev_init=0.001,
                      hidden_act='relu', outlayer_act='prelu', weight_decay=0.001, width=360, depth=3):
    """ Fonction créant un réseau de neurones de type fully connected, ayant une couche d'entrée de 360
    neurones, quatre couches cachées de 360 neurones et une sortie de 240 neurones
    Inputs : hyperparamètres
    """

    # On créé l'initialisateur de tenseur avec une loi normale tronquée. sigma = stddev_init, et les 
    # valeurs à plus de 2sigma sont re-tirées
    winit = tfl.initializations.truncated_normal(stddev=stddev_init, dtype=tf.float32, seed=None)
    
    # On créé l'input du RN
    network = input_data(shape=[None, 360], name='input')
    
    # On créé les couches cachées
    for i in range(depth):
        network = fully_connected(network, width, activation=hidden_act, name='fc'+str(i), weights_init=winit,
                                  weight_decay=weight_decay)
        # On détruit des neurones aléatoirement avec une la probabilité donnée en entrée
        network = dropout(network, dropout_val)
    
    # On ajoute la couche de sortie du réseau
    # Fonction d'activation prelu
    # Initilisée avec la loi normale tronquée
    network = fully_connected(network, 240, activation=outlayer_act, name='outlayer', weights_init=winit)
    
    adam = Adam(learning_rate=learning_rate, epsilon=epsilon)
    
    # Couche d'évaluation du modèle. Utilisation d'une descente stochastique Adam
    # Learning rate = 0.05
    # Loss = fonction définie rmse
    network = regression(network, optimizer=adam,
    loss=partial_rmse, metric=partial_r2_score, name='target')
            
    return network

/home/etudiant/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/etudiant/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Instructions for updating:
Use the retry module or similar alternatives.


## Préparation des données


#### Fonction renvoyant deux sous-ensembles du jeu d'entrainement : un ensemble d'exemples et les cibles correspondantes


In [5]:
def get_fold(train_set, targets, reduce_train_fold_size):
    """ Permet d'obtenir un sous-ensemble du jeu d'entraînement afin de ne pas travailler sur le jeu
    d'entraînement total pour la recherche par quadrillage et donc de gagner du temps d'exécution. L'idée
    et que si un ensemble d'hyper-paramètres produit des meilleurs résultats que les autres ensembles
    d'hyper-paramètres sur l'ensemble du jeu d'entraînement, alors on suppose que ce sera également 
    le cas sur une partie des données. """

    return (train_set["inputs"][:reduce_train_fold_size], targets["targets"][:reduce_train_fold_size])
    

## Entraînement des modèles

#### Fonction d'entraînement du modèle

In [16]:
import h5py
import tflearn as tfl
import time
from scipy import sparse
import numpy as np
import gc
import tensorflow as tf


def train_model(input_X, labels_y, model_name, model_path, logs_path, samples_per_batch=1000, epochs=5,
                learning_rate=0.001, epsilon=1e-8, dropout=0.99, stddev_init=0.001, hidden_act='relu',
                outlayer_act='prelu'):
    
    total_start_time = time.time()

    tf.reset_default_graph()
    
    # On créé le réseau 
    network = creer_360_x_240(learning_rate=learning_rate, epsilon=epsilon, dropout_val=dropout,
                                 stddev_init=stddev_init, hidden_act=hidden_act, outlayer_act=outlayer_act)

    # On créé le modèle
    model = tfl.DNN(network, tensorboard_verbose=3, tensorboard_dir=logs_path)

    # Entraînement
    model.fit(X_inputs=input_X,Y_targets=labels_y, batch_size=samples_per_batch,
              shuffle = True, snapshot_step=100, validation_set=0.1,
              show_metric=True, run_id=model_name, n_epoch=epochs)

    # Sauvegarde du modèle
    model.save(model_path + model_name + ".tflearn")


#### Entraînement du modèle

In [15]:
import h5py


input_X_h5 = h5py.File(train_prepared_input_loc, 'r')
labels_y_h5 = h5py.File(train_labels_loc, 'r')

"""
input_X_h5 = h5py.File(mini_prepared_input_loc, 'r')
labels_y_h5 = h5py.File(mini_labels_loc, 'r')

"""
input_X, labels_y = get_fold(input_X_h5, labels_y_h5, 2500000)

model_name = "DELTA_DIST+H_05_basic"
model_path = models_loc
logs_path = logs_loc

train_model(input_X, labels_y, model_name, model_path, logs_path, samples_per_batch=10000, 
            epochs=5, learning_rate=0.01, dropout=0.97, epsilon=0.001, hidden_act="relu6",
            outlayer_act="linear")


[ -63.81879   -60.495556  -27.063326 -124.909164  323.44885   311.92914
  262.79605   353.49997  -200.99821   -53.761936 -329.13626  -172.72247
  -83.03802  -144.36479  -189.3064    132.93494  -269.8073   -308.40668
 -224.28836  -269.59384    67.13267    65.626045   38.904064   94.60527
 -110.683586 -120.240555 -116.269165 -195.38763    43.11627   -45.85881
   15.567158   13.33028  -103.34256  -116.60969   -67.16978   -73.85322
 -410.7601   -405.34308  -393.58395  -359.42197  -119.461395  -80.16686
 -136.78775   -80.40891    19.101686   22.287884  -75.83423    28.024733
  -16.571526   -9.863982  -11.059247   19.688393  180.58093   180.54265
  167.05176   184.57639   210.31181   181.68173   156.72606   199.78143
 -189.90836  -225.70856  -183.54733   -88.49711    28.604616   64.47135
  -15.137252  -17.497793   21.275692   -7.958674   17.721409   39.20046
    0.          0.          0.          0.          0.          0.
    0.          0.          0.          0.          0.          0.
 

/home/etudiant/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


---------------------------------
Run id: DELTA_DIST+H_05_basic
Log directory: ../models/DELTA_DIST+H_05/8.3/
INFO:tensorflow:Summary name partial_r2/ (raw) is illegal; using partial_r2/__raw_ instead.
---------------------------------
Training samples: 1
Validation samples: 1
--
Training Step: 1  | time: 1.452s
| Adam | epoch: 001 | loss: 0.00000 - partial_r2/r2_score: 0.0000 | val_loss: 195.43549 - val_acc: 0.8524 -- iter: 1/1
--
